In [ ]:
import numpy as np
import scio
from data import Data
import metadatabase as mdb
import matplotlib.pyplot as plt

In [ ]:
def scio_length(file_name):

    # Read file as a string.
    try:
        data = scio._read_file_as_string(file_name)
    except:
        return 0

    # Catches empty files.
    if len(data) == 0:
        return 0

    # Get the file dimension.
    i = 0;
    dimension = np.abs(np.frombuffer(data[i:i+4], dtype='int32')[0])
    i = i + 4

    # Extract the number of frequenct channels.
    channels = np.frombuffer(data[i:i+4*dimension], dtype='int32')

    # Get the data type.
    i = i + 4*dimension
    data_type = np.frombuffer(data[i:i+4], dtype='int32')[0]
    i = i + 4

    # File size sanity check.
    bytes_per_frame = scio.int2nbyte(data_type)*np.product(channels)
    number_of_bytes = len(data) - i
    remainder = np.remainder(number_of_bytes, bytes_per_frame)

    return number_of_bytes/bytes_per_frame

In [ ]:
categories=['Antenna', 'Switch']
instruments=['70MHz']
channels=['EW']
intervals=[(1554200000.0, 1554300000.0)]

In [ ]:
d = Data.via_metadatabase(categories=categories,
                          instruments=instruments,
                          channels=channels,
                          intervals=intervals,
                          quality=[1, 'NULL'],
                          integrity=[1, 0, 'NULL'],
                          completeness=[1, 0, 'NULL'])

In [ ]:
paths = mdb.locate(categories=categories,
                   instruments=instruments,
                   channels=channels,
                   intervals=intervals,
                   quality=[1, 0, 'NULL'],
                   integrity=[1, 0, 'NULL'],
                   completeness=[1, 0, 'NULL'])

pol_files = [file_path for (_, _, file_path, _, file_name, _) in paths if 'pol' in file_name]
pol_lines = np.array([scio_length('/project/s/sievers/prizm' + pol_file) for pol_file in pol_files])

In [ ]:
plt.figure(1, figsize=(100, 100))
plt.title('pol')
plt.imshow(np.log10(d.get(data='pol', instrument=instruments[0], channel=channels[0], partition=None)), vmin=4.5, vmax=8.5, aspect=1)
plt.hlines(np.cumsum(pol_lines), np.zeros_like(pol_lines), 4095*np.ones_like(pol_lines), color='black', alpha=0.1, linewidth=1)
plt.hlines(np.cumsum(pol_lines)[[0]], 0, 4095, color='red', alpha=1.0, linewidth=1)

In [ ]:
plt.figure(1, figsize=(100,100))
plt.plot(d.get(data='time_sys_start', instrument=instruments[0], channel=channels[0], partition=None), '-', markersize=1)
plt.vlines(np.cumsum(pol_lines), d[instruments[0]][channels[0]]['time_sys_start'][0], np.max(d[instruments[0]][channels[0]]['time_sys_start']), color='black', linewidth=1, alpha=0.1)
plt.ylim(d[instruments[0]][channels[0]]['time_sys_start'][0], np.max(d[instruments[0]][channels[0]]['time_sys_start']))


plt.vlines(np.cumsum(pol_lines)[[0]], d[instruments[0]][channels[0]]['time_sys_start'][0], np.max(d[instruments[0]][channels[0]]['time_sys_start']), color='red', linewidth=1, zorder=100)

for (flag, ctime) in d[instruments[0]]['Switch']['antenna']:
    plt.hlines(ctime, 0, d[instruments[0]][channels[0]]['time_sys_start'].shape[0], color='blue', linewidth=1)

In [ ]:
pol_files[0]

In [ ]:
pol_files[6]

In [ ]:
d.partition(instruments=['70MHz'], channels=['EW'], buffer=(0,0))
plt.figure(1, figsize=(100, 100))
plt.title('pol')
plt.imshow(np.log10(d.get(data='pol', instrument=instruments[0], channel=channels[0], partition='antenna')), vmin=4.5, vmax=8.5, aspect=1)